In [ ]:
#!pip3 install tqdm
from post_processing import *
from const import ROOT
from const import *
from PIL import ImageOps
def tint_image(src, color="#FFFFFF"):
    src.load()
    r, g, b, alpha = src.split()
    gray = ImageOps.grayscale(src)
    result = ImageOps.colorize(gray, (0, 0, 0, 0), color) 
    result.putalpha(alpha)
    return result
import pandas as pd
#ROOT='/Users/alexanderliao/data/'
ROOT='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/'
#ROOT='/data/kaggle/salt/'
train_path = os.path.join(ROOT, 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]
test_path = os.path.join(ROOT, 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]


def is_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def down_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(50,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[50,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def find_cols(img):
    row = img[100,:]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def find_rows(img):
    row = img[:,0]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def vertical_sweep(img,col1,col2):
    img2=img.copy()
    img2[:,col1:col2+1]=1
    return img2

def vertical_clear(img,col1,col2):
    img2=img.copy()
    img2[:,0:col1]=0
    img2[:,col2:]=0
    return img2

def horizontal_clear(img,col1,col2):
    img2=img.copy()
    img2[0:col1,:]=0
    img2[col2:,:]=0
    return img2

def ul_empty(img1,img2):
    if img1[:,0:10].sum()==0 and img2[0:10,:].sum()==0: return True
    return False
    
def end_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(90,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[90,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def up_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(25):
        if img[i,:].sum()==0: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0
dicts = pickle.load( open( "linked_list.p", "rb" ) )
def id_to_img(id):
    if id in train_file_list:
        str="train"
    else:
        str="test"
    img = Image.open(ROOT+str+"/images/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr

def id_to_mask(id):
    img = Image.open(ROOT+"train/masks/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr
import tensorflow as tf
df = pd.read_csv("0871.csv")
def column(matrix, i):
    return [row[i] for row in matrix]
def rule_empty1(col_curr,j):
    return id_to_img(col_curr[j-1]).sum()!=0
def isnt_empty(img):
    return img.sum()>0 and img.sum()<2600000
def rule_ne(col_curr,j):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def deep_rejection(col_curr,j,vert_mask):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def rle_decode(rle_mask,size):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(size*size, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(size,size)

def get_iou_vector(A, B):
    # Numpy version
    
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def iou_metric(label, pred):
    # Tensorflow version
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)

def LB_improvement(new_code, old_code):
    strs=""
    for string in new_code:
       strs=strs+str(string)+" " 
    img_new = np.transpose(rle_decode(strs,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def LB_improvement2(new_code, old_code):
    img_new = np.transpose(rle_decode(new_code,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def discrepancy(img):
    diffs=[]
    #if img.sum()==0: return False
    for i in range(101):
        #if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    return discrepancy

In [ ]:
#df = pd.read_csv("final/ocnet256_resnet256_0450350101_0871empty_leak4++_stage2+++.csv")

In [ ]:
dfne = pd.read_csv("final/ocnet256_resnet256_0450350101.csv")
#df.sort_values(by=['id'])
#remove= [1,5,9,13,15,18,21,25,29,33,43,45,57,62,65,74,107,110,136,150,7,42]
remove = range(1,151)

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3++.csv")
#df = pd.read_csv("test.csv")
RULE1=True
RULE2=True
RULE3=True
RULE4=True
RULE5=True
rule1=0
rule2=0
rule3=0
rule4=0
ct=0
t1=True
per_im=0
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if t1 and isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if len(row)==2 and len(col_curr)>=7:
                    imgs=[]
                    imgs.append(list[0][0])
                    imgs.append(list[0][1])
                    imgs.append(list[len(col_curr)-1][0])
                    imgs.append(list[len(col_curr)-1][1])
                    """
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    col_r = column(list,row.index(col)-1)
                    imgs.append(col_r[index1-1])
                    imgs.append(col_r[index1-2])
                    imgs.append(col_curr[index1+3])
                    imgs.append(col_curr[index1+4])
                    """
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()!=0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    #cand = col
                    if logic1:
                        for row1 in list:
                            for col in row:
                                col_curr = column(list,row.index(col))
                                index1 = col_curr.index(col)
                                index2 = row.index(col)
                                if(index1<3 or index1 >len(col_curr)-4):
                                    cand=list[index1][index2]
                                    if cand in test_file_list:
                                        df.loc[df['id'] == cand,'rle_mask']=""
                                        ct=ct+1
                                        per_im=4
                                        rule4=rule4+1
                                        visited.append(col)
                                        #t1=False
                                        print("-----------")
                                        print("rule1")
                                        print("-----------")
                                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))

#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
#remove= [1,5,9,13,15,18,21,25,29,33,43,45,57,62,65,74,107,110,136,150,7,42,4]
#        y y y  y  y  y  y  y y  y  y   y  y  y  y y   y   y   y   y  n  y
extra=[20,34,39,47]
other_half=[13,43,65,110,136,150,7,42,4]

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv")
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if t1 and isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if len(row)==2 and len(col_curr)==6:
                    imgs=[]
                    imgs.append(list[0][0])
                    imgs.append(list[0][1])
                    imgs.append(list[len(col_curr)-1][0])
                    imgs.append(list[len(col_curr)-1][1])
                    """
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    col_r = column(list,row.index(col)-1)
                    imgs.append(col_r[index1-1])
                    imgs.append(col_r[index1-2])
                    imgs.append(col_curr[index1+3])
                    imgs.append(col_curr[index1+4])
                    """
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()!=0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    #cand = col
                    if logic1:
                        for row1 in list:
                            for col in row:
                                col_curr = column(list,row.index(col))
                                index1 = col_curr.index(col)
                                index2 = row.index(col)
                                if(index1<3 or index1 >len(col_curr)-3):
                                    cand=list[index1][index2]
                                    if cand in test_file_list:
                                        df.loc[df['id'] == cand,'rle_mask']=""
                                        ct=ct+1
                                        per_im=4
                                        rule4=rule4+1
                                        visited.append(col)
                                        #t1=False
                                        print("-----------")
                                        print("rule1")
                                        print("-----------")
                                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))

#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv")
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if t1 and isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if index2==1 and index1 >0 and index1 < len(col_curr)-2 and len(row)==3 and len(col_curr)>=6:
                    imgs=[]
                    
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    imgs.append(col_curr[index1+1])
                    imgs.append(col_curr[index1+2])
                    imgs.append(row[index2-1])
                    imgs.append(row[index2+1])
                    #col_r = column(list,row.index(col)-1)
                    #imgs.append(col_r[index1-1])
                    #imgs.append(col_r[index1-2])
                    #imgs.append(col_curr[index1+3])
                    #imgs.append(col_curr[index1+4])
                    
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()!=0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    cand = col
                    if logic1 and cand in test_file_list:
                        df.loc[df['id'] == cand,'rle_mask']=""
                        ct=ct+1
                        per_im=per_im+1
                        rule4=rule4+1
                        visited.append(col)
                        
                        print("-----------")
                        print("empty up + right")
                        print("-----------")
                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
                        
#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv")
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if index2==len(row)-1 and index1 >0 and index1 < len(col_curr)-3 and len(row)>4 and len(col_curr)>7:
                    imgs=[]
                    
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    imgs.append(col_curr[index1-3])
                    imgs.append(col_curr[index1-4])
                    imgs.append(col_curr[index1+1])
                    imgs.append(col_curr[index1+2])
                    imgs.append(col_curr[index1+3])
                    #imgs.append(row[index2-1])
                    #imgs.append(row[index2+1])
                    #col_r = column(list,row.index(col)-1)
                    #imgs.append(col_r[index1-1])
                    #imgs.append(col_r[index1-2])
                    #imgs.append(col_curr[index1+3])
                    #imgs.append(col_curr[index1+4])
                    
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()!=0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    cand = col
                    if logic1 and cand in test_file_list:
                        df.loc[df['id'] == cand,'rle_mask']=""
                        ct=ct+1
                        per_im=per_im+1
                        rule4=rule4+1
                        visited.append(col)
                        
                        print("-----------")
                        print("empty up + right")
                        print("-----------")
                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
                        
#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
a=np.array([0,1,1,1,1,1,1,0,1,1,0])
rle_encoding(a)

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv")
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if index2==1 and index1==1  and len(row)>4 and len(col_curr)>=6:
                    imgs=[]
                    
                    imgs.append(col_curr[index1-1])
                    col_left=column(list,row.index(col)-1)
                    imgs.append(col_left[index1-1])
                    imgs.append(col_left[index1+0])
                    imgs.append(col_left[index1+1])
                    imgs.append(col_left[index1+2])
                    imgs.append(col_left[index1+3])
                    
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    cand = col
                    if logic1 and cand in test_file_list:
                        df.loc[df['id'] == cand,'rle_mask']=""
                        ct=ct+1
                        per_im=per_im+1
                        rule4=rule4+1
                        visited.append(col)
                        
                        print("-----------")
                        print("empty up + right")
                        print("-----------")
                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
                        
#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv")
for i in remove:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
           #========================================================================================#

            if isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
#========================================================================================#

                if index2>2 and index2 < len(row)/2 and index1==2  and len(row)>4 and len(col_curr)>=6:
                    imgs=[]
                    
                    imgs.append(col_curr[index1-2])
                    col_left=column(list,row.index(col)-1)
                    imgs.append(col_left[index1-1])
                    imgs.append(col_left[index1+0])
                    imgs.append(col_left[index1+1])
                    imgs.append(col_left[index1+2])
                    imgs.append(col_left[index1+3])
                    col_right=column(list,row.index(col)+1)
                    imgs.append(col_l[index1-1])
                    imgs.append(col_left[index1+0])
                    imgs.append(col_left[index1+1])
                    imgs.append(col_left[index1+2])
                    imgs.append(col_left[index1+3])
                    
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                    """
                    if i==1 and index1==5 and index2==12:
                        print(index2)
                        print(masks)
                    """
                    cand = col
                    if logic1 and cand in test_file_list:
                        df.loc[df['id'] == cand,'rle_mask']=""
                        ct=ct+1
                        per_im=per_im+1
                        rule4=rule4+1
                        visited.append(col)
                        
                        print("-----------")
                        print("empty up + right")
                        print("-----------")
                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                        print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
                        
#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)

In [ ]:
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+++.csv",index=False)
#df.to_csv("final/test.csv",index=False)
other_half=[13,43,65,110,136,150,7,42,4]

In [ ]:
add=[1,5,6,10,12,13,14,18,19,23,33,34,46,53,56,58,67,77,78,81,102,106]

In [ ]:
remove2=[]